# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing ~~graphlab~~ functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use ~~SFrames~~ pandas to do some feature engineering
* Use built-in ~~graphlab~~ functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

## Constants

In [29]:
# import graphlab as glab

import version_information
import pandas as pd
import os
import numpy as np
import math
import sklearn
from sklearn import linear_model as lm

In [30]:
%load_ext version_information
%version_information pandas, numpy, math, os, sklearn

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 2.7.12 64bit [MSC v.1500 64 bit (AMD64)]
IPython 5.1.0
OS Windows 10 10.0.14393
pandas 0.19.1
numpy 1.11.2
math The 'math' distribution was not found and is required by the application
os The 'os' distribution was not found and is required by the application
sklearn 0.18.1
Tue Dec 13 22:46:43 2016 Eastern Standard Time

## Get Data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [31]:
os.getcwd()

'C:\\Users\\DrjNupurVihan\\Documents\\GitHub\\MachineLearning_UoW\\Course2_Regression'

In [32]:
# sales = graphlab.SFrame('kc_house_data.gl/')
sales = pd.read_csv("data\\kc_house_train_data.csv")
train_data = pd.read_csv("data\\kc_house_train_data.csv")
test_data = pd.read_csv("data\\kc_house_test_data.csv")

In [33]:
train_data.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [34]:
train_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [35]:
#Split data into training and testing.

#We use seed=0 so that everyone running this notebook gets the same results.
#In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

# train_data,test_data = sales.random_split(.8,seed=0)

## Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data.

In [36]:
example_features = train_data.ix[:,['sqft_living', 'bedrooms', 'bathrooms']]
print example_features.head()

output = train_data.price

   sqft_living  bedrooms  bathrooms
0         1180         3       1.00
1         2570         3       2.25
2          770         2       1.00
3         1960         4       3.00
4         1680         3       2.00


In [37]:
# example_model = glab.linear_regression.create(train_data, target = 'price', features = example_features, validation_set = None)
#(Aside: We set validation_set = None to ensure that the results are always the same)


example_model = lm.LinearRegression()
mreg_model = example_model.fit(example_features, output)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [38]:
# example_weight_summary = example_model.get("coefficients")
mreg_model.coef_
# print example_weight_summary

array([   315.40669062, -65081.88711588,   6942.16598637])

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [39]:
# example_predictions = example_model.predict(train_data)
mreg_model.predict(example_features)[0]
# print example_predictions[0] # should be 271789.505878 (rounding errors may be present of not using graphlab)

271789.26537996985

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [40]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    y_pred = model.predict(data)

    # Then compute the residuals/errors
    errors = y_pred - outcome

    # Then square and add them up
    RSS = (errors ** 2).sum() 

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [41]:
# rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
test_features = test_data.ix[:,['sqft_living', 'bedrooms', 'bathrooms']]
rss_example_test = get_residual_sum_of_squares(mreg_model, test_features, test_data['price'])
print rss_example_test # should be 2.7376153833e+14 (rounding errors may be present of not using graphlab)

2.73761940583e+14


## Creating new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = $bedrooms^2$
* bed_bath_rooms = $bedrooms*bathrooms$
* log_sqft_living = $log$(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [42]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [43]:
# creating the remaining 3 features in both TEST and TRAIN data

train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['log_sqft_living'] = train_data['sqft_living'].apply(math.log)
test_data['log_sqft_living'] = test_data['sqft_living'].apply(math.log)

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since $1^2 = 1$ but $4^2 = 16$. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

### Quiz Question 1, 2, 3, 4: 

What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)

In [44]:
print round(test_data['bedrooms_squared'].mean(),2)
print round(test_data['bed_bath_rooms'].mean(),2)
print round(test_data['log_sqft_living'].mean(),2)
print round(test_data['lat_plus_long'].mean(),2)

12.45
7.5
7.55
-74.65


## Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:

* Model 1: $squarefeet$, $bedrooms$, $bathrooms$, $latitude$ & $longitude$
* Model 2: Add $bedrooms*bathrooms$  to model 1  parameters
* Model 3: Add $log(squarefeet)$, $bedrooms^2$, and the (nonsensical) $latitude + longitude$

In [45]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

In [46]:
X_model_1 = train_data.ix[:, model_1_features].copy()
X_model_2 = train_data.ix[:, model_2_features].copy()
X_model_3 = train_data.ix[:, model_3_features].copy()

In [47]:
Y = train_data.price

Now that you have the features, learn the weights for the three different models for predicting target = 'price' ~~using graphlab.linear_regression.create()~~ and look at the value of the weights/coefficients. We also want to evaluate which model is best for model comparison purposes.

In [48]:
# Learn the model 1 (don't forget to set validation_set = None if using graphlab)
model_1 = example_model.fit(X_model_1, Y)
rss_model_1_train = get_residual_sum_of_squares(model_1, X_model_1, Y)
print model_1.coef_
print rss_model_1_train

[  3.12258646e+02  -5.95865332e+04   1.57067421e+04   6.58619264e+05
  -3.09374351e+05]
9.6787996305e+14


In [49]:
# Learn the model 2 (don't forget to set validation_set = None if using graphlab)
model_2 = example_model.fit(X_model_2, Y)
rss_model_2_train = get_residual_sum_of_squares(model_2, X_model_2, Y)
print model_2.coef_
print rss_model_2_train

[  3.06610053e+02  -1.13446368e+05  -7.14613083e+04   6.54844630e+05
  -2.94298969e+05   2.55796520e+04]
9.58419635074e+14


In [50]:
# Learn the model 3 (don't forget to set validation_set = None if using graphlab)
model_3 = example_model.fit(X_model_3, Y)
rss_model_3_train = get_residual_sum_of_squares(model_3, X_model_3, Y)
print model_3.coef_
print rss_model_3_train

[  5.29422820e+02   3.45142296e+04   6.70607813e+04   5.34085611e+05
  -4.06750711e+05  -8.57050439e+03  -6.78858667e+03  -5.61831484e+05
   1.27334900e+05]
9.0343645505e+14


### Quiz Question 5: 

What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?

In [51]:
print model_1_features

# We note that bathrooms is the third feature in the list, 
# So we need to see the sign of the 3rd coefficient in the above list of model 1 coeffs (and is positive)

['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']


### Quiz Question 6: 

What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?

In [52]:
print model_2_features

# We note that bathrooms is the third feature in the list, 
# So we need to see the sign of the 3rd coefficient in the above list of model 2 coeffs (and is negative)

['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms']


## Comparing multiple models

### Quiz Question 7

Which model (1, 2 or 3) has lowest RSS on TRAINING Data?

**ANS:** Based on the rss values from training data set, it can be seen that model 3 has the lowest RSS.

Now compute the RSS on on TEST data for each of the three models.

### Quiz Question 8: 

Which model (1, 2 or 3) has lowest RSS on TESTING Data?

In [53]:
X_model_1_test = test_data.ix[:, model_1_features].copy()
X_model_2_test = test_data.ix[:, model_2_features].copy()
X_model_3_test = test_data.ix[:, model_3_features].copy()
Y_test = test_data.price

In [54]:
# RSS from the model 1
model_1_test = example_model.fit(X_model_1_test, Y_test)
rss_model_1_test = get_residual_sum_of_squares(model_1_test, X_model_1_test, Y_test)
print rss_model_1_test

2.23588470558e+14


In [55]:
# RSS from the model 2
model_2_test = example_model.fit(X_model_2_test, Y_test)
rss_model_2_test = get_residual_sum_of_squares(model_2_test, X_model_2_test, Y_test)
print rss_model_2_test

2.20888287428e+14


In [56]:
# RSS from the model 3
model_3_test = example_model.fit(X_model_3_test, Y_test)
rss_model_3_test = get_residual_sum_of_squares(model_3_test, X_model_3_test, Y_test)
print rss_model_3_test

2.11457519132e+14
